In [1]:
!pip install transformers[torch] datasets pandas torch accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [3]:
# Load the dataset
from google.colab import files
uploaded = files.upload()


Saving your_dataset.csv to your_dataset.csv


In [4]:
# Assuming the uploaded file is named 'your_dataset_file.csv'
file_path = next(iter(uploaded))

In [5]:
# Try reading the file with different encodings and handle parsing errors
encodings = ['utf-8', 'latin1', 'iso-8859-1']
data = None
for encoding in encodings:
    try:
        data = pd.read_csv(file_path, encoding=encoding, delimiter=',', quotechar='"')
        break
    except UnicodeDecodeError:
        continue
    except pd.errors.ParserError:
        data = pd.read_csv(file_path, encoding=encoding, delimiter=';', quotechar='"')
        break

if data is None:
    raise ValueError("Unable to read the CSV file with the specified encodings and delimiters.")

In [6]:
# Keep only the second and third columns
data = data.iloc[:, [1, 2]]
data.columns = ['Raw Content', 'Intriguers']
# Check for null values
data.dropna(subset=['Raw Content', 'Intriguers'], inplace=True)


# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(data)

<ipython-input-6-39d15a40da04>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(subset=['Raw Content', 'Intriguers'], inplace=True)


In [7]:
print(data.head)

<bound method NDFrame.head of                                             Raw Content  \
0     Every year, nature puts on a breathtaking disp...   
1     Every year, nature puts on a breathtaking disp...   
2     Every year, nature puts on a breathtaking disp...   
3     Every year, nature puts on a breathtaking disp...   
4     The Lantern Festival in Taiwan, celebrated on ...   
...                                                 ...   
1272  The Nest Learning Thermostat is a smart thermo...   
1273  The Nest Learning Thermostat is a smart thermo...   
1274  The Nest Learning Thermostat is a smart thermo...   
1275  The Nest Learning Thermostat is a smart thermo...   
1276  The Nest Learning Thermostat is a smart thermo...   

                                             Intriguers  
0     Ever thought of a springtime adventure where y...  
1     How about a unique spring festival next April?...  
2     Think about the unforgettable moments you coul...  
3     Imagine capturing the p

In [8]:
# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(data)

In [9]:
# Load pre-trained model and tokenizer
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
# Preprocess the data
def preprocess_data(examples):
    inputs = [ex for ex in examples['Raw Content']]
    targets = [ex for ex in examples['Intriguers']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length').input_ids
    model_inputs['labels'] = labels
    return model_inputs

In [11]:
# Apply the preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/1277 [00:00<?, ? examples/s]

In [12]:
# Split the dataset into train and eval sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [13]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    # Enable GPU usage
    fp16=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


In [15]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,0.120244
1,No log,0.084225
2,No log,0.074018
4,1.260200,0.067627
5,1.260200,0.065758
6,0.080500,0.064538
8,0.080500,0.063226
9,0.080500,0.063104


TrainOutput(global_step=1430, training_loss=0.490932955441775, metrics={'train_runtime': 1099.4637, 'train_samples_per_second': 10.451, 'train_steps_per_second': 1.301, 'total_flos': 6960997646991360.0, 'train_loss': 0.490932955441775, 'epoch': 9.947826086956521})

In [16]:
# Define the directory where you want to save the trained model
output_model_dir = './saved_model/'

# Create the directory if it doesn't exist
import os
os.makedirs(output_model_dir, exist_ok=True)

# Save the model to the specified directory
model.save_pretrained(output_model_dir)

# Optionally, save the tokenizer as well if needed
tokenizer.save_pretrained(output_model_dir)

print(f"Model saved to {output_model_dir}")

Model saved to ./saved_model/


In [17]:
# Directory where your model and tokenizer are saved
output_model_dir = './saved_model/'

# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(output_model_dir)
tokenizer = T5Tokenizer.from_pretrained(output_model_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# Example of a test input
test_input = """Virat Kohli, one of cricket's greatest modern batsmen, is known for his aggressive style and remarkable consistency. With numerous records under his belt, including the fastest century in ODIs for India, Kohli's passion and dedication to the game are evident every time he steps onto the field. His leadership skills and on-field performance have earned him accolades and a massive fan following worldwide"""

# Tokenize the input
inputs = tokenizer(test_input, return_tensors="pt")

In [19]:
# Generate multiple outputs using top-k sampling
num_outputs = 5
outputs = model.generate(
    inputs['input_ids'],
    max_length=50,
    num_beams=5,
    num_return_sequences=num_outputs,
    early_stopping=True,
    do_sample=True,        # Enable sampling
    top_k=50,              # Top-k sampling
    top_p=0.95             # Top-p (nucleus) sampling
)

# Decode the outputs to text
predicted_outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Print the outputs
for i, output in enumerate(predicted_outputs, 1):
    print(f"Output {i}: {output}")

Output 1: Known for his aggressive style and remarkable consistency, Virat Kohli is one of cricket's greatest modern batsmen.
Output 2: Known for his aggressive style and remarkable consistency, Virat Kohli is known for his on-field performance.
Output 3: Known for his aggressive style and remarkable consistency, Virat Kohli is known for his commitment to the game.
Output 4: Virat Kohli is known for his aggressive style and remarkable consistency.
Output 5: Known for his aggressive style and remarkable consistency, Virat Kohli is known for his on-field performance and leadership skills.


In [20]:
!zip -r saved_model.zip saved_model

  adding: saved_model/ (stored 0%)
  adding: saved_model/spiece.model (deflated 48%)
  adding: saved_model/model.safetensors (deflated 9%)
  adding: saved_model/generation_config.json (deflated 29%)
  adding: saved_model/config.json (deflated 63%)
  adding: saved_model/special_tokens_map.json (deflated 85%)
  adding: saved_model/added_tokens.json (deflated 83%)
  adding: saved_model/tokenizer_config.json (deflated 94%)


In [21]:
from google.colab import files
files.download('saved_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>